## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kaitangata,NZ,2022-05-25 20:13:49,-46.2817,169.8464,40.03,82,96,7.92
1,1,Conceicao Do Araguaia,BR,2022-05-25 20:13:49,-8.2578,-49.2647,84.92,56,4,2.46
2,2,Awjilah,LY,2022-05-25 20:13:49,29.1081,21.2869,88.27,9,0,9.31
3,3,Carnarvon,AU,2022-05-25 20:13:50,-24.8667,113.6333,71.24,65,100,8.79
4,4,Port Alfred,ZA,2022-05-25 20:13:42,-33.5906,26.8910,62.19,82,0,7.00


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Carnarvon,AU,2022-05-25 20:13:50,-24.8667,113.6333,71.24,65,100,8.79
5,5,Puerto Ayora,EC,2022-05-25 20:13:50,-0.7393,-90.3518,76.96,90,67,5.99
8,8,Bengkulu,ID,2022-05-25 20:13:51,-3.8004,102.2655,74.21,87,100,4.97
9,9,Albany,US,2022-05-25 20:09:23,42.6001,-73.9662,76.44,47,99,5.66
11,11,Rikitea,PF,2022-05-25 20:13:51,-23.1203,-134.9692,75.00,76,21,6.53
13,13,Alofi,NU,2022-05-25 20:13:51,-19.0595,-169.9187,78.69,83,40,8.05
15,15,Hirara,JP,2022-05-25 20:13:52,24.8000,125.2833,75.18,91,100,4.36
18,18,Jamestown,US,2022-05-25 20:13:08,42.0970,-79.2353,75.02,54,20,11.01
26,26,Namatanai,PG,2022-05-25 20:13:53,-3.6667,152.4333,78.39,80,100,4.41
35,35,Altay,CN,2022-05-25 20:13:55,47.8667,88.1167,76.91,25,100,3.13


In [9]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

209


City_ID       209
City          209
Country       209
Date          209
Lat           209
Lng           209
Max Temp      209
Humidity      209
Cloudiness    209
Wind Speed    209
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

209


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Carnarvon,AU,2022-05-25 20:13:50,-24.8667,113.6333,71.24,65,100,8.79
5,5,Puerto Ayora,EC,2022-05-25 20:13:50,-0.7393,-90.3518,76.96,90,67,5.99
8,8,Bengkulu,ID,2022-05-25 20:13:51,-3.8004,102.2655,74.21,87,100,4.97
9,9,Albany,US,2022-05-25 20:09:23,42.6001,-73.9662,76.44,47,99,5.66
11,11,Rikitea,PF,2022-05-25 20:13:51,-23.1203,-134.9692,75.00,76,21,6.53


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Carnarvon,AU,71.24,-24.8667,113.6333,
5,Puerto Ayora,EC,76.96,-0.7393,-90.3518,
8,Bengkulu,ID,74.21,-3.8004,102.2655,
9,Albany,US,76.44,42.6001,-73.9662,
11,Rikitea,PF,75.00,-23.1203,-134.9692,
13,Alofi,NU,78.69,-19.0595,-169.9187,
15,Hirara,JP,75.18,24.8000,125.2833,
18,Jamestown,US,75.02,42.0970,-79.2353,
26,Namatanai,PG,78.39,-3.6667,152.4333,
35,Altay,CN,76.91,47.8667,88.1167,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.loc[hotel_df["Hotel Name"] == "", [Hotel Name]] = np.nan
clean_hotel_df = hotel_df.dropna(axis="index", how="any")

print (clean_hotel_df.count())
clean_hotel_df.head(10)

City          209
Country       209
Max Temp      209
Lat           209
Lng           209
Hotel Name    209
dtype: int64


,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Carnarvon,AU,71.24,-24.8667,113.6333,Hospitality Carnarvon
5,Puerto Ayora,EC,76.96,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Bengkulu,ID,74.21,-3.8004,102.2655,Grage Hotel Bengkulu
9,Albany,US,76.44,42.6001,-73.9662,
11,Rikitea,PF,75.00,-23.1203,-134.9692,People ThankYou
13,Alofi,NU,78.69,-19.0595,-169.9187,Taloa Heights
15,Hirara,JP,75.18,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
18,Jamestown,US,75.02,42.0970,-79.2353,Hampton Inn & Suites Jamestown
26,Namatanai,PG,78.39,-3.6667,152.4333,Seagulls Inn Namatani
35,Altay,CN,76.91,47.8667,88.1167,Jinqiao Hotel


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv" 

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))